# JSON Retriever  
Create a chain to retrieve information from json by making a request to a web page

In [2]:
import json
import requests

from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.llms import Ollama

In [3]:
# create a json saver to get response and save as json
def save_response_as_json(url, filename):
    response = requests.get(url)
    data = response.json()
    with open(filename, 'w') as f:
        json.dump(data, f)

In [6]:
url = "https://jsonplaceholder.typicode.com/posts/1"
path = "response.json"

In [7]:
save_response_as_json(url, path)

In [11]:
# create json loader without any specific schema
loader = JSONLoader(
    file_path=path,
    jq_schema='.',
    text_content=False)

In [12]:
# load json
data = loader.load()

In [ ]:
# create vectorstore from data
embeddings = OllamaEmbeddings(model="llama3")
vectorStore = Chroma.from_documents(data, embeddings)

In [ ]:
# create retriever to get related information
retriever = vectorStore.as_retriever(search_kwargs={"k": 1}) # the retriever uses similarity_search, which has a default value of  n_results k=4

In [ ]:
# LLM
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
ollama_llm = Ollama(
    model = 'llama3',
    verbose= False,
    callback_manager=callback_manager
    )

In [ ]:
# create propt template to feed the llm
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template= """You're an asistant that is talking to a customer. You are retrieving responses in json format as results of requests.
    Answer only the questions about the documents and contexts you have. If you don't know the answer,
    say that you don't know. Keep your replies short. Don't hallucinate, give only a simple answer.

    Contexts:{context}
    History: {history}

    User: {question}
    Chatbot:"""
)

In [ ]:
# create memory agent to save the chat history
memory = ConversationBufferMemory(
       memory_key="history",
       return_messages=True,
       input_key="question",
   )

In [ ]:
while True:
    user_input = input("You:")
    docs = retriever.invoke(user_input)
    chain = prompt | ollama_llm
    if user_input.lower() in ["bye", "exit"]:
        break
    chain.invoke({
        "question": user_input,
        "history": memory,
        "context": docs  # "contexts": "\n---\n".join([d.page_content for d in docs])
        }
    )
    print("\n")